In [1]:
import os
import pandas as pd
import re
import logging
from tqdm import tqdm

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
def clean_body(content):
    """
    Limpia el contenido del cuerpo del correo eliminando saltos de línea (\n)
    y tabulaciones (\t), dejando solo espacios.
    """
    content = content.replace("\n", " ").replace("\t", " ").strip()
    content = re.sub(r"\s+", " ", content)  # Reemplazar múltiples espacios por uno solo
    return content


In [4]:
def extract_email_parts(file_content):
    """
    Extrae las partes 'From', 'Subject' y el 'Body' del archivo.
    Si 'Subject' está ausente, el 'Body' comienza inmediatamente después del 'From'.
    """
    # Buscar la línea que contiene "From:"
    from_match = re.search(r"From:\s*(.*)", file_content)
    
    # Buscar la línea que contiene "Subject:" (puede contener diferentes prefijos)
    subject_match = re.search(r"Subject:\s*(.*)", file_content, re.IGNORECASE)
    
    # Inicializamos el contenido de las partes
    from_part = None
    subject_part = None
    body_part = None
    
    # Si encontramos "From"
    if from_match:
        from_part = from_match.group(1).strip()
        
        # Si encontramos también el "Subject"
        if subject_match:
            subject_part = subject_match.group(1).strip()
            
            # El Body comienza después del primer salto de línea tras el Subject
            body_start = file_content.find("\n", subject_match.end())
        else:
            # Si no hay Subject, el Body comienza justo después del "From"
            body_start = file_content.find("\n", from_match.end())
        
        # Extraer el body desde el punto donde empieza
        body_part = file_content[body_start:].strip()

        # Asegurarse de que el `From` no está duplicado en el Body
        body_part = body_part.replace(f"From: {from_part}", "").strip()

        # Limpiar el body eliminando \n y \t
        body_part = clean_body(body_part)
    
    return from_part, subject_part, body_part

In [5]:

def process_files_in_folder(folder_path: str = "data/20N"):
    """
    Procesa todos los archivos dentro de la carpeta y extrae la información
    'From', 'Subject' y 'Body' para crear un DataFrame.
    """
    data = []
    
    # Obtener la lista de archivos
    total_folders = len([folder for folder in os.listdir(folder_path) if folder != "__init__.py"])
    
    for folder in os.listdir(folder_path):
        if folder != "__init__.py":
            folder_full_path = os.path.join(folder_path, folder)
            files = [file for file in os.listdir(folder_full_path) if file != "__init__.py"]
            total_files = len(files)
            
            # Usar tqdm para mostrar la barra de progreso
            for file in tqdm(files, desc="Processing files", unit="file"):
                with open(os.path.join(folder_full_path, file), 'r') as f:
                    file_content = f.read()

                    # Extraer las partes del archivo
                    from_part, subject_part, body_part = extract_email_parts(file_content)

                    if from_part and body_part:  # Subject es opcional
                        data.append({
                            "category": folder,
                            "email_id": file,
                            'from': from_part,
                            'subject': subject_part,
                            'body': body_part
                        })

                # Calcular el porcentaje completado y cuánto falta
                completed_files = len(data)
                percentage_complete = (completed_files / total_files) * 100
                remaining_files = total_files - completed_files
                
                # Registrar el progreso
                logger.info(f"Progreso: {percentage_complete:.2f}% completado. Restan {remaining_files} archivos.")

    # Convertir la lista de diccionarios en un DataFrame
    df = pd.DataFrame(data)
    return df

In [6]:
df_emails = process_files_in_folder()

Processing files:   0%|          | 0/799 [00:00<?, ?file/s]

2024-09-07 17:43:48,986 - INFO - Progreso: 0.13% completado. Restan 798 archivos.
2024-09-07 17:43:48,986 - INFO - Progreso: 0.25% completado. Restan 797 archivos.
2024-09-07 17:43:48,999 - INFO - Progreso: 0.38% completado. Restan 796 archivos.
2024-09-07 17:43:49,006 - INFO - Progreso: 0.50% completado. Restan 795 archivos.
2024-09-07 17:43:49,006 - INFO - Progreso: 0.63% completado. Restan 794 archivos.
2024-09-07 17:43:49,016 - INFO - Progreso: 0.75% completado. Restan 793 archivos.
2024-09-07 17:43:49,021 - INFO - Progreso: 0.88% completado. Restan 792 archivos.
2024-09-07 17:43:49,022 - INFO - Progreso: 1.00% completado. Restan 791 archivos.
2024-09-07 17:43:49,028 - INFO - Progreso: 1.13% completado. Restan 790 archivos.
2024-09-07 17:43:49,034 - INFO - Progreso: 1.25% completado. Restan 789 archivos.
2024-09-07 17:43:49,034 - INFO - Progreso: 1.38% completado. Restan 788 archivos.
2024-09-07 17:43:49,040 - INFO - Progreso: 1.50% completado. Restan 787 archivos.
2024-09-07 17:43

In [8]:
df_emails.to_csv("20N_unified.csv")